Resources used:
- https://docs.aws.amazon.com/autoscaling/ec2/userguide/GettingStartedTutorial.html


Notes:
* It is better to use launch template than launch configuration

`put_scaling_policy` will create CloudWatch alarm.

In [10]:
import boto3
from mypy_boto3_autoscaling import AutoScalingClient
from mypy_boto3_cloudwatch import CloudWatchClient
from pprint import pprint


In [2]:
shell_script = open("script.sh", "r").read()
print(shell_script)


#!/bin/bash

yum update -y
yum install -y httpd
service httpd start
service enable httpd

cd /var/www/html

# Download data from S3
wget https://ujjwal-bucket-boto.s3.amazonaws.com/files.zip
unzip files.zip
rm files.zip


In [3]:
autoscaling_client: AutoScalingClient = boto3.client(
    "autoscaling", region_name="us-east-1"
)

autoscaling_group_name: str = "Assignment 3"
launch_template_id: str = "lt-095f31ed6be2a5d8e"


In [6]:
autoscaling_client.create_auto_scaling_group(
    AutoScalingGroupName=autoscaling_group_name,
    MinSize=1,
    MaxSize=3,
    DesiredCapacity=2,
    LaunchTemplate={"LaunchTemplateId": launch_template_id},
    AvailabilityZones=["us-east-1b"],
)


{'ResponseMetadata': {'RequestId': '11bf1a6b-7d7a-4f98-8063-cc585a3a4d1e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '11bf1a6b-7d7a-4f98-8063-cc585a3a4d1e',
   'content-type': 'text/xml',
   'content-length': '231',
   'date': 'Wed, 08 Sep 2021 04:17:00 GMT'},
  'RetryAttempts': 0}}

In [12]:
scale_up_policy = autoscaling_client.put_scaling_policy(
    AutoScalingGroupName=autoscaling_group_name,
    PolicyName="Scale up",
    PolicyType="SimpleScaling",
    AdjustmentType="ChangeInCapacity",
    ScalingAdjustment=1,
)

pprint(scale_up_policy)


{'Alarms': [],
 'PolicyARN': 'arn:aws:autoscaling:us-east-1:602291782684:scalingPolicy:d2fd9e95-31bb-45fe-bbe8-d68fd60b68af:autoScalingGroupName/Assignment '
              '3:policyName/Scale up',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '463',
                                      'content-type': 'text/xml',
                                      'date': 'Wed, 08 Sep 2021 05:07:05 GMT',
                                      'x-amzn-requestid': '4feade6f-9423-4893-8292-308ad47193fe'},
                      'HTTPStatusCode': 200,
                      'RequestId': '4feade6f-9423-4893-8292-308ad47193fe',
                      'RetryAttempts': 0}}


In [13]:
scale_down_policy = autoscaling_client.put_scaling_policy(
    AutoScalingGroupName=autoscaling_group_name,
    PolicyName="Scale down",
    # PolicyType="SimpleScaling",
    AdjustmentType="ChangeInCapacity",
    ScalingAdjustment=-1,
)

pprint(scale_down_policy)


{'Alarms': [],
 'PolicyARN': 'arn:aws:autoscaling:us-east-1:602291782684:scalingPolicy:42d282dc-d012-4a19-8190-cb5512e39b07:autoScalingGroupName/Assignment '
              '3:policyName/Scale down',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '465',
                                      'content-type': 'text/xml',
                                      'date': 'Wed, 08 Sep 2021 05:07:09 GMT',
                                      'x-amzn-requestid': 'dcc5ad66-d568-4ba4-8c9c-05c9fd3780f3'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'dcc5ad66-d568-4ba4-8c9c-05c9fd3780f3',
                      'RetryAttempts': 0}}


In [25]:
cloudwatch_client: CloudWatchClient = boto3.client("cloudwatch")

cloudwatch_client.put_metric_alarm(
    AlarmName="EC2 Scale Up",
    EvaluationPeriods=1,
    ComparisonOperator="GreaterThanThreshold",
    MetricName="CPUUtilization",
    AlarmActions=[scale_up_policy["PolicyARN"]],
    Threshold=60,
    Period=60,
    Namespace="AWS/EC2",
    Statistic="Average",
)


{'ResponseMetadata': {'RequestId': 'dfa23f2c-9983-4c62-8c01-20ebfa2b3769',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'dfa23f2c-9983-4c62-8c01-20ebfa2b3769',
   'content-type': 'text/xml',
   'content-length': '214',
   'date': 'Wed, 08 Sep 2021 05:16:16 GMT'},
  'RetryAttempts': 0}}

In [26]:
cloudwatch_client.put_metric_alarm(
    AlarmName="EC2 Scale Down",
    EvaluationPeriods=1,
    ComparisonOperator="LessThanThreshold",
    MetricName="CPUUtilization",
    AlarmActions=[scale_down_policy["PolicyARN"]],
    Threshold=60,
    Period=60,
    Namespace="AWS/EC2",
    Statistic="Average",
)


{'ResponseMetadata': {'RequestId': 'e2f041cd-bace-4472-9d02-36ca548e750a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e2f041cd-bace-4472-9d02-36ca548e750a',
   'content-type': 'text/xml',
   'content-length': '214',
   'date': 'Wed, 08 Sep 2021 05:16:18 GMT'},
  'RetryAttempts': 0}}